# Overview
This notebook contains the implementation of the Multivariate Sea Storm Model (https://github.com/anardek/Multivariate_Sea_Storm_Model/tree/main) used to generate 20,000 synthetic storms parameterized for North Core Banks, NC using a 30-year historical time record. Below, I provide a summary of the MSSM components and use within Barrier3D (taken from the notebook located here: XXXX). The storms developed here are used within the manuscript " " Van Blunk et al., (in prep).

# Storms in Barrier3D
Storms in Barrier3D are be described by 3 variables: storm duration, the highest runup elevation (Rhigh), and the lowest runup elevation (Rlow). In this notebook, we describe how to generate a list of synthetic storms for use in Barrier3D following the methods of Wahl et al. (2016) for a copula-based multivariate sea-storm model (MSSM). In this notebook, the MSSM model is developed using wave hindcast and water-level data from the North Core Banks, NC and accounts for interdependencies between these three variables. We then illustrate how the list of synthetic storms can be used to generate annual storm time series (i.e., many storms per model year) for use in Barrier3D. Most of the text below describing the MSSM is taken from the Supplementary Text of Reeves et al., 2021 and the notebook here: XXXX).

## Multivariate Sea Storm Model
The MSSM model `multivariateSeaStorm.m` (located at https://github.com/anardek/Multivariate_Sea_Storm_Model) was written in Matlab in order to utilize the t-tide package, which allows for robust fitting of tidal constituents to water level time series, and therefore sadly cannot be run in this notebook. We provide a sample call below and describe the function components: 

`[stStorms, stSimStorms] = multivariateSeaStorm(sCopula, sWIS_filename, sWaterLevel_filename, fBeta, fBermEl, nSimStorm, bPlot, sOutput_filename)`

The MSSM model requires the following inputs: 

`
% Inputs:
%       sCopula              - copula to be fitted to storm variables:"c-vine", "d-vine", "gaussian"
%       sWIS_filename        - .onlns file downloaded from a USACE Wave Information Studies (WIS) bouy; must 
%                              contain hourly records of wave height (m)
%       sWaterLevel_filename - .txt file that contains hourly records of total water level in m NAVD88 as second 
%                              column, first column is datetime; downloaded for each year from NOAA;
%                              must be either the same length or longer than WIS time record
%       fBeta                - beach slope 
%       nSimStorm            - number of simulated storms to create
%       fBermEl              - erosion threshold; Wahl used 5% of dune toe heights; we use the average berm 
%                              elevation (m NAVD88)
%       bPlot                - boolean for plotting
%       sOutputFilename      - string of prefix for csv output filename
`

For North Core Banks, we utilize a 30 year record of hourly wave hindcast data – including wave height (Hs) and wave period (Tp) – from the USACE’s Wave Information Studies buoy offshore North Core Banks (ST63266, 24 m water depth) and hourly records of water level from the nearest NOAA tide gauge (Station 8656483, Beaufort, NC) to create a list of 20,000 synthetic storms. The data range used spans June 10, 1990 to June 10, 2021 (GMT/UTC); the Beaufort, NC tide gauge was installed in June 1990 and the WIS ONELINES time series extends through 2021. Note that the time series from the NOAA tide gauge were downloaded at year-long intervals, combined into a single CSV, and then a custom datetime was created ('dd-mmm-yyyy hh:mm:ss') prior to exporting as a txt file. I also had to manually delete NANs from the water level time series (January 1992).

<img align="left" width="500" height="300" src="FIGS/WIS.png"> 
<img align="right" width="450" height="600" src="FIGS/WIS_output.png"> 
<img align="left" width="500" height="600" src="FIGS/TidesCurrents.png">

We specify a berm elevation of 1.46 m NAVD88 and beach slope of 0.03 (both values represent an average for the northern end of North Core Banks, calculated using USACE LiDAR from 2018). The c-vine produced the highest $\tau$ values (Kendall's Corelation Coefficient) -- over the elliptical Gaussian and T-student copulas, as well as the d-vine copula -- and is therefore shown here. [say we get 788 storms using a wave height threshold of 1.75 m]

`[stStorms, stSimStorms] = multivariateSeaStorm("c-vine", "ST63266-time_series-061090_061021.onlns", "CO-OPS_8656483_COMBINED_Reduced.txt", 0.03, 1.46, 20000, true, "StormList_20k_NCB_Berm1pt46m_Slope0pt03.csv", "/Users/KatherineAnardeWheels/PycharmProjects/CASCADE/notebooks/synthetic_storm_creation_NCB/data", 1990)`

<img align="left" width="600" height="600" src="Fig1-QC-VCR_Berm1pt9m_Slope0pt04.png">  

Within `multivariateSeaStorm.m`, we first process the data by removing the 365-day (1 yr) running median. This differs from the 30-day running median used in Wahl et al. (2016), namely because we desired to maintain seasonal trends and only account for non-stationarity in wave and water level parameters due to inter-annual and decadal variability. The median of the last 3 years is then applied to the entire time series such that the new time series is representative of the current climate. A year-by-year tidal analysis is performed using t_Tide (Pawlowicz et al., 2002) to obtain the tidal amplitude $\eta_A$ and non-tidal residual $\eta_{NTR}$. Lastly, the representative highest elevation of the landward margin of runup (Rhigh, i.e. the total water level) is calculated as the sum of the maximum 2% exceedance of runup, following Stockdon et al. (2006), and the contemporaneous (corrected) water level elevation from the tide gauge. The representative lowest runup elevation (Rlow), below which any part of the barrier is considered continuously subaqueous during a storm, is taken as Rhigh – swash/2 (Figure 1). While wave direction is processed as part of the MSSM, it is not required as input for Barrier3D and is therefore not discussed herein.

### Figure 1. Timeseries corrected for non-stationarity due to inter-annual and decadal variability 